# Operator splitting 

In [1]:
# Importing relevant libraries 
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 
from nm_lib import nm_lib as nm
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# plt.style.use('dark_background')
plt.style.use('default')

def u(x): 
    """ 
    Initial condition for t=t0 when a = const

    Parameters
    ----------
    x : `array`
        Spatial axis. 
    
    Returns
    -------
    Equation 2
    """
    return np.cos(6*np.pi*x / 5)**2 / np.cosh(5*x**2)

x0 = -2.6
xf = 2.6
a = -0.7
b = -0.3

## 1- OS precision

Solve the following Burgers' equation: 

$$\frac{\partial u}{\partial t} = - a \frac{\partial u}{\partial x} - b \frac{\partial u}{\partial x}   \tag{1}$$

following exersize [2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb). where $x[x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, $a=-0.7$ and $b=-0.3$, periodic boundary conditions and with initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2)  \tag{2}$$

Solve the evolution for the following four different approaches: 

- 1 With additive OS.  

- 2 With Lie-Trotter OS. 

- 3 With Strang OS.

- 4 Without an operator splitting and single time-step method but add the to terms: 

$$\frac{\partial u}{\partial t} = - (a+b) \frac{\partial u}{\partial x}$$

for $nump=256$ and 100 steps.

_Suggestion_: use the Lax-method scheme for all cases with `deriv_cent`. Make sure the boundaries are properly selected.

Fill in the function `osp_LL_Add`, `osp_LL_Lie`, and `osp_LL_Strang`.

Start with $cfl\_cut = 0.4$ and increase up to $0.9$.  

Which OS schemes are stable? Which one is more diffusive? Why?

In [3]:
nump = 256
nt = 100*2
xx = np.arange(nump)/(nump-1.0) * (xf-x0) + x0

t_lax, unnt_lax = nm.evolv_Lax_adv_burgers(xx, u(xx), nt, a+b,  cfl_cut=0.4, ddx=nm.deriv_cent, bnd_limits=[1,1])
t_add, unnt_add = nm.ops_Lax_LL_Add(       xx, u(xx), nt, a, b, cfl_cut=0.4, ddx=nm.deriv_cent, bnd_limits=[1,1])
t_lie, unnt_lie = nm.ops_Lax_LL_Lie(       xx, u(xx), nt, a, b, cfl_cut=0.4, ddx=nm.deriv_cent, bnd_limits=[1,1])
t_LLS, unnt_LLS = nm.ops_Lax_LL_Strang(    xx, u(xx), nt, a, b, cfl_cut=0.4, ddx=nm.deriv_cent, bnd_limits=[1,1])

# Find the common elements in the four arrays
common1 = np.where(np.isclose(t_lax, t_add, atol=5e-0))[0]
common2 = np.where(np.isclose(t_lie, t_LLS, atol=5e-0))[0]
common_times = np.where(np.isclose(common1, common1, atol=5e-0))[0]
t = t_lax[common_times]

# Animation 
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
N = 10

def init(): 
    axes.plot(xx, unnt_lax[:,0])
    axes.plot(xx, unnt_add[:,0])
    axes.plot(xx, unnt_lie[:,0])
    axes.plot(xx, unnt_LLS[:,0])

def animate(i):
    axes.clear()
    # axes.plot(xx, unnt_lax[:,::N][:,common_times[i]], label='LL lax')
    # axes.plot(xx, unnt_add[:,::N][:,common_times[i]], label='LL add')
    # axes.plot(xx, unnt_lie[:,::N][:,common_times[i]], label='LL lie')
    # axes.plot(xx, unnt_LLS[:,::N][:,common_times[i]], label='LL strang')

    axes.plot(xx,unnt_add[:,i], label='add')
    index_lie = np.argmin(np.abs(t_add[i]-t_lie))
    axes.plot(xx,unnt_lie[:,index_lie], label='Lie')
    index_strang = np.argmin(np.abs(t_add[i]-t_LLS))
    axes.plot(xx,unnt_LLS[:,index_strang], label='LL_Strang')
    index_ab = np.argmin(np.abs(t_add[i]-t_lax))
    axes.plot(xx,unnt_lax[:,index_ab], label='Lax (a+b)')
    axes.set_title('t≈%.2f,%.2f,%.2f,%.2f' %(t_lie[index_lie], t_add[i], t_LLS[index_strang], t_lax[index_ab]))

    axes.legend()
    axes.set_ylim((0, 1.01))
    # axes.set_title('t=%.2f' %t[::N][i])
    axes.grid(True)

anim = FuncAnimation(fig, animate, interval=50, frames=len(t[::N]), init_func=init)
plt.close()
HTML(anim.to_jshtml())

<span style="color:pink">

For $cfl\_cut = 0.4$ we see that the additive OS blows up after a few timesteps. The other methods do now blow up ,with the Strang OS being somewhat more diffusive than the Lie-Trotter OS. 

The additive OS blows up faster when using $cfl\_cut = 0.9$, but the Lie-Trotter OS looks like it is less diffusive in this case. Both of these methods are first order, which probably is what is causing this. 

The Lax method, without operator splitting, is less diffusive than the other methods in both cases, but is slower. 
</span>

<span style="color:green">JMS</span>.

<span style="color:Blue">God except for the Lax</span>.

<span style="color:red">In fact the problem is that the timestep is not the same. In order to compare apples to apples you need to select the closet instance for the one of the OS respect to the others (similar to the problem you have in 4c).  </span>.


## 2- When does it not work? 

Use OS-Strang from the previous exercise and try to apply a predictor-corrector explicit method. 
To facilitate this exercise, `nm_lib` already includes the predictor-corrector Hyman method, which is included Bifrost (`Hyman`). Fill in the function `osp_Lax_LH_Strang`. Use the same setup as the previous exercise but with $nump=512$, $500$ steps, and $cfl\_cut=0.8$. 

What do you notice? 

__Optional__: Apply the Hyman predictor-corrector explicit method to the Burgers equation and check if the following is true: 

$$u^{n+1} = F\, u^{n}\Delta t \approx G\, u^{n}\Delta t+H\, u^{n}\Delta t$$

In [4]:
nump = 512
nt = 500
xx = np.arange(nump)/(nump-1.0) * (xf-x0) + x0

t_lax, unnt_lax = nm.evolv_Lax_adv_burgers(xx, u(xx), nt, a+b,  cfl_cut=0.8, ddx=nm.deriv_cent,bnd_limits=[1,1])
t_LHS, unnt_LHS = nm.osp_Lax_LH_Strang(    xx, u(xx), nt, a, b, cfl_cut=0.8, ddx=nm.deriv_cent,bnd_limits=[1,1])
t_LLS, unnt_LLS = nm.ops_Lax_LL_Strang(    xx, u(xx), nt, a, b, cfl_cut=0.8, ddx=nm.deriv_cent,bnd_limits=[1,1])

# Find the common elements in the four arrays
common1 = np.where(np.isclose(t_LHS, t_LLS, atol=5e1))[0]
common_times = np.where(np.isclose(common1, t_lax, atol=5e1))[0]
t = t_lax[common_times]

# Animation 
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
N = 10

def init(): 
    # axes.plot(xx, unnt_lax[:,common_times[0]])
    axes.plot(xx, unnt_LHS[:,0])
    axes.plot(xx, unnt_LLS[:,0])

def animate(i):
    axes.clear()

    axes.plot(xx, unnt_LHS[:,i], label='add')
    index_LLS = np.argmin(np.abs(t_LHS[i]-t_LLS))
    axes.plot(xx,unnt_LLS[:,index_LLS], label='LL_Strang')

    axes.set_title('t≈%.2f,%.2f' %(t_LLS[index_LLS], t_LHS[i]))

    # # axes.plot(xx, unnt_lax[:,::N][:,common_times[i]], label='LL lax')
    # axes.plot(xx, unnt_LHS[:,::N][:,i], label='LL lie')
    # index_LL = np.argmin(np.abs(t_LHS[i] - t_LLS))
    # axes.plot(xx, unnt_LLS[:,::N][:,index_LL], label='LL strang')
    axes.legend()
    axes.set_ylim((0, 1.01))
    # axes.set_title('t=%.2f, %.2f'%t_LLS[::N][index_LL], t_LHS[::N][i])
    axes.grid(True)
anim = FuncAnimation(fig, animate, interval=1, frames=len(t[::N]), init_func=init)
plt.close()
HTML(anim.to_jshtml())

<span style="color:pink">

The LH Strang method and OS-Strang methods behave similarly, with LH Strang being a little more diffusive, and a bit slower. Compared to the Lax method, both are faster and more diffusive. 

</span>

<span style="color:green">JMS</span>.

<span style="color:red">As mentioned above, Lax looks strange </span>.

<span style="color:red">In addition, Strang and LH Strang should be different. See my comments above</span>.